In [ ]:
from IPython.display import HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Reconocimiento de objetos en imágenes









**Ariel Rossanigo**


### Quien soy?

* Ariel Rossanigo
* Profe de Inteligencia Artificial
* Developer, Data Scientist


### Objetivos de la charla

* Mostrar como podemos atacar este tipo de problemas
* Que todos se vayan con una idea del pipeline y que hace cada paso




### Agenda

* Definición del problema
* Código
* Explicación de las distintas partes involucradas

### Distintos tipos de problemas en CV

#### Clasificación

<img src="./imgs/mnist.jpg" width="600" align="middle">


### Distintos tipos de problemas en CV

#### Clasificación: Algunas veces no es tan simple

<img src="./imgs/dogs_vs_bread.jpg" width="600" align="middle">


#### Localización

<img src="./imgs/cat_face_detector_result_01.jpg" width="600" align="middle">

https://www.pyimagesearch.com/2016/06/20/detecting-cats-in-images-with-opencv/

#### Detección de objetos

<img src="./imgs/object_detection.png" width="600" align="middle">

 https://arxiv.org/abs/1311.2524

#### Segmentación

<img src="./imgs/segmentation.png" width="600" align="middle">

https://arxiv.org/abs/1703.06870

<em style="float: right;">10</em>

### Una solución *"Out of the box"*

Google ofrece una serie de modelos implementados en ``TensorFlow``, entre ellos uno que permite detectar objetos.

Se puede descargar desde acá:

https://github.com/tensorflow/models/tree/master/research/object_detection

Tiene un tutorial para usarlo *"de fábrica"*, y otro para para entrenar nuestro propio reconocedor de objetos.


In [ ]:
%matplotlib inline
import numpy as np
import os
import tensorflow as tf
from matplotlib import pyplot as plt

from object_detection_helpers import open_image, load_labels, visualize_image

#### Bajamos el modelo y configuramos algunos paths

El modelo se puede bajar desde 

http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_11_06_2017.tar.gz


In [ ]:
base_path = 'object_detection'
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
PATH_TO_CKPT = os.path.join('object_detection', MODEL_NAME, 'frozen_inference_graph.pb')

# Leemos el modelo que bajamos de internet
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

# Leemos las clases con las que fue entrenado el modelo
category_index = load_labels(os.path.join(base_path, 'data', 'mscoco_label_map.pbtxt'), 90)

#### Leemos algunas imágenes...

In [ ]:
TEST_IMAGE_PATHS = [os.path.join('test_images', x) for x in os.listdir('test_images')]

In [ ]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for image_path in TEST_IMAGE_PATHS:

            image_np = open_image(image_path)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
             
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
              [boxes, scores, classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})

            visualize_image(image_np, boxes, classes, scores, category_index)

#### Datos en crudo

In [ ]:
scores
# visualize_image(image_np.copy(), boxes, classes, scores, category_index, min_score_thresh=0.01)

<em style="float: right;">20</em>

### Los bloques

<div><img src="imgs/bloques.jpg" width="400" style="float: right;"></div>

<div class="alert alert-block alert-info">Clasificador de imágenes</div>

<div class="alert alert-block alert-info">Detector / proponedor de regiones</div>



### Redes neuronales

<div><img src="imgs/neuron.jpeg" width="400" style="float: left; margin: 10px;"></div>

<div><img src="imgs/neural_network.jpg" width="400" style="float: right; margin: 10px;"></div>

<div style="clear:both;"></div>
http://cs231n.github.io/neural-networks-1/

### Redes neuronales: Noción básica de funcionamiento

<div><img src="imgs/mnist_input.png" width="400" align="middle" style=" margin: 10px;" ></div>


https://codelabs.developers.google.com/codelabs/cloud-tensorflow-mnist/#0

In [ ]:
%matplotlib inline
from object_detection_helpers import get_mnist_demo, show_weights
model, X, y = get_mnist_demo()

In [ ]:
show_weights(model)

In [ ]:
for i in range(6):
    model.fit(X, y, epochs=1)
    show_weights(model)

### Redes neuronales: Noción básica de funcionamiento

<div><img src="imgs/mnist_multilayer.png" width="500" align="middle" style="margin: 10px;"></div>

https://www.youtube.com/watch?v=aircAruvnKk

### Deep Learning

*The hierarchy of concepts enables the computer to learn complicated concepts bybuilding them out of simpler ones. If we draw a graph showing how these concepts are built on top of each other, the graph is deep, with many layers. For this reason,we call this approach to AI deep learning.*


**Goodfellow et al 2016**

<em style="float: right;">30</em>

### Convoluciones

<div><img src="imgs/conv_explanation.jpg" width="50%" style="float: left;"></div>

https://developer.apple.com/library/content/documentation/Performance/Conceptual/vImage/ConvolutionOperations/ConvolutionOperations.html

### Convoluciones


<div><img src="imgs/applying_filter.gif" width="80%" style="float: left; margin: 10px;"></div>

https://www.kdnuggets.com/2016/11/intuitive-explanation-convolutional-neural-networks.html/2

### Convoluciones

#### Parámetros más comunes

* **filters:** cantidad de filtros
* **kernel_size:** tamaño del kernel
* **strides:** la cantidad de pasos que muevo el kernel
* **padding:** agrega ceros en los bordes para mantener el tamaño original
* **activation:** función de activación aplicada ``pixel a pixel``


### Pooling

<div><img src="imgs/max_pooling.png" width="50%" style="float: left;"></div>

* reduce el tamaño de los filtros
* brinda robustez 
* features *equivariantes*

<div style="clear:both;"></div>

http://textminingonline.com/dive-into-tensorflow-part-v-deep-mnist

### Redes neuronales

<div><img src="imgs/neural_network_complete.png" width="90%" style="float: left;"></div>

https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/comment-page-2/

<em style="float: right;">40</em>

### Tenemos un clasificador, ¿y ahora qué?

### Sliding windows

<div><img src="imgs/sliding_window.gif" width="90%" style="float: left; margin: 10px;" ></div>

https://matthewearl.github.io/2016/05/06/cnn-anpr/

### 2014: R-CNN (Region Convolutional Neural Network)

<div><img src="imgs/rcnn.png" width="80%" style="float: none; margin: 10px;"  align="middle" ></div>

https://arxiv.org/abs/1311.2524

1. Selective search para proponer regiones (~2000 por imagen)
2. AlexNet -> SVM para clasificar
3. Linear regression para mejorar el box

### 2015: Fast R-CNN

<div><img src="imgs/fast_rcnn.png" width="70%" style="margin: 10px;"  align="middle" ></div>

https://arxiv.org/abs/1504.08083

### 2016: Faster R-CNN 

<div><img src="imgs/faster_rcnn.png" height="60%" style="margin: 10px;"  align="middle" ></div>

https://arxiv.org/abs/1506.01497

### 2016: SSD (Single Shot Detector)

<div><img src="imgs/ssd.png" width="90%" style="margin: 10px;"  align="middle" ></div>

https://arxiv.org/abs/1512.02325

<em style="float: right;">50</em>

### Resumen para atacar un problema

* Probar solución *Out of the box*
* Re-entrenar el modelo para que se ajuste a las clases del problema
* Elegir el modelo en base a las necesidades de tiempo y precisión que tengamos



### Preguntas?

<img src="imgs/man-qmark.jpg" width="400" align="middle">


### Gracias!

Mis datos de contacto:

<p><img src="imgs/gmail-1162901_960_720.png" width="40" style="float: left;" align="middle"> arielrossanigo@gmail.com</p>

<p><img src="imgs/twitter-312464_960_720.png" width="40" style="float: left;" align="middle"> @arielrossanigo</p>

<p><img src="imgs/github-154769__340.png" width="40" style="float: left;" align="middle"> https://github.com/arielrossanigo</p>
